# Уровень смертности от непреднамеренного отравления (на 100 000 населения)

In [1]:
import pandas as pd

from warnings import simplefilter
simplefilter('ignore')

Эти данные можно найти на сайте ВОЗ по ссылке \
https://www.who.int/data/gho/data/themes/topics/indicator-groups/poison-control-and-unintentional-poisoning

In [2]:
# Загрузим таблицу
data = pd.read_csv('../data/who_poisoning_mortality.csv')

data.head(2)

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,SDGPOISON,Mortality rate attributed to unintentional poi...,numeric,WPR,Western Pacific,Country,BRN,Brunei Darussalam,Year,2019,...,NaN,NaN,0.0005,NaN,0.002,0.0 [0.0-0.0],NaN,NaN,EN,2021-02-08T21:00:00.000Z
1,SDGPOISON,Mortality rate attributed to unintentional poi...,numeric,AMR,Americas,Country,VCT,Saint Vincent and the Grenadines,Year,2019,...,NaN,NaN,0.0030,NaN,0.008,0.0 [0.0-0.0],NaN,NaN,EN,2021-02-08T21:00:00.000Z


Описание нужных нам столбцов.

- `SpatialDimValueCode` - код страны
- `Period` - год
- `FactValueNumeric` - уровень смертности от непреднамеренного отравления (на 100 000 населения)

Оставим только их.

In [3]:
data = data[['SpatialDimValueCode', 'Period', 'FactValueNumeric']]

# Переименуем столбец FactValueNumeric в PoisoningMortality
data.rename(
    columns={'FactValueNumeric': 'PoisoningMortality'},
    inplace=True,
)

data.head()

,SpatialDimValueCode,Period,PoisoningMortality
0,BRN,2019,0.001
1,VCT,2019,0.005
2,SGP,2019,0.019
3,ISR,2019,0.047
4,MDV,2019,0.047


In [4]:
# Убедимся, что нет пропусков
data.isna().sum().sum()

0

In [5]:
print(f"Диапазон лет: {data['Period'].min()} - {data['Period'].max()} гг.")

Диапазон лет: 2000 - 2019 гг.


Данные за 2020 год заполним средним за последние 3 года по стране.

In [6]:
# Маска для последних 3-х лет
mask = data['Period'].isin([2019, 2018, 2017])

# Сгруппируем по коду страны и возьмем среднее
mean_last_3_years_data = data[mask].groupby('SpatialDimValueCode').agg('mean')

# Сбросим индекс
mean_last_3_years_data.reset_index(inplace=True)

# Проставим 2020 год
mean_last_3_years_data['Period'] = 2020

mean_last_3_years_data.head()

,SpatialDimValueCode,Period,PoisoningMortality
0,AFG,2020,1.020000
1,AGO,2020,2.053333
2,ALB,2020,0.320000
3,ARE,2020,0.350000
4,ARG,2020,0.470000


Соединим данные за 2020 год с предыдущей таблицей.

In [7]:
data = pd.concat([
    data,
    mean_last_3_years_data,
], ignore_index=True)

data.head()

,SpatialDimValueCode,Period,PoisoningMortality
0,BRN,2019,0.001
1,VCT,2019,0.005
2,SGP,2019,0.019
3,ISR,2019,0.047
4,MDV,2019,0.047


In [8]:
# Сохраним полученную таблицу для дальнейшего использования
data.to_csv('../data/who_poisoning_mortality_prepared.csv', index=False)